In [ ]:
import requests
import json
from secrets import ASTROBIN_KEY, ASTROBIN_SECRET

In [ ]:
astrobin_url = 'http://www.astrobin.com'
api_url = '/api/v1/image/'
base_params = {'api_key': ASTROBIN_KEY, 'api_secret': ASTROBIN_SECRET}
data_dir = 'data/'
jupiter_dir = data_dir+'jupiter/'

In [ ]:
def search(queries):
    results = []
    for query in queries:
        query_results = []
        print 'Querying... ' + str(query)
        query.update(base_params)
        search_request = requests.get(astrobin_url+api_url, params=query)
        if search_request.status_code == 200:
            page_results = json.loads(search_request.text)
            print 'Attempting to fetch ' + str(page_results['meta']['total_count']) + ' results...'
            query_results.extend(page_results['objects'])
            while page_results['meta']['next']:
                search_request = requests.get(astrobin_url+page_results['meta']['next'])
                if search_request.status_code == 200:
                    page_results = json.loads(search_request.text)
                    query_results.extend(page_results['objects'])
                    print page_results['meta']['offset']
                else:
                    print 'NEXT PAGE FAILED: ' + str(page_results['meta']['offset'])
            
            print 'No more pages!'
            
        else:
            print 'SEARCH FAILED'
            print query
        
        print 'Fetched ' + str(len(query_results)) + ' results for query ' + str(query)
        results.extend(query_results)
    
    print 'Fetched ' + str(len(results)) + ' results total.'
    return results

In [ ]:
def build_metadata(results):
    metadata = {}
    for result in results:
        if result['id'] not in metadata:
            metadata[result['id']] = result
    
    print 'Found ' + str(len(metadata.keys())) + ' unique metadata entries for ' + str(len(results)) + ' results.'
    return metadata

In [ ]:
def download_image(details, image_dir=''):
    image_request = requests.get(details['url_real'])
    if image_request.status_code == 200:
        with open(data_dir+image_dir+details['id']+'.jpg', 'wb') as image:
            image.write(image_request.content)
    else:
        print 'IMAGE REQUEST FAILED: ' + details['id']

In [ ]:
jupiter_results = search([{'title__icontains': 'jupiter'}, {'description__icontains': 'jupiter'}])

In [ ]:
with open(jupiter_dir+'jupiter_results.json','w') as outfile:
    json.dump(jupiter_results, outfile)

In [ ]:
jupiter_metadata = build_metadata(jupiter_results)

In [ ]:
with open(jupiter_dir+'jupiter_metadata.json', 'w') as outfile:
    json.dump(jupiter_metadata, outfile)